In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders import get_preprocessing_fn
import os
import wandb
import yaml

In [ ]:
run_type = 'sky-eye-full' #'building-damage'#

conf_file = {'sky-eye-full': 'config-seg.yaml', 'building-damage': 'config-damage.yaml'}[run_type]

os.environ['WANDB_MODE'] = 'dryrun'
wandb.init(project=run_type, config = yaml.load(open(conf_file)))
conf = wandb.config

In [ ]:
from pprint import pprint
pprint(dict(conf))

In [ ]:
from glob import glob
from tqdm import tqdm_notebook as tqdm
import numpy as np
import torch
from torch import nn
from xv.util import vis_im_mask
import segmentation_models_pytorch as smp

In [ ]:
train_dir = '/home/jupyter/datasets/xview/train'
test_dir = '/home/jupyter/datasets/xview/test'

In [ ]:
import albumentations as al

augment = al.Compose([
        al.HorizontalFlip(p=conf.aug_prob),
        al.VerticalFlip(p=conf.aug_prob),
        al.RandomRotate90(p=conf.aug_prob),
        al.Transpose(p=conf.aug_prob),
        al.GridDistortion(p=conf.aug_prob, distort_limit=.2),
        al.ShiftScaleRotate(p=conf.aug_prob),
        al.RandomBrightnessContrast(p=conf.aug_prob)
])

In [ ]:
from xv.nn.nets import DownscaleLayer, XVNet
import segmentation_models_pytorch as smp

segmentation_types = {
    'PSPNet': smp.PSPNet,
    'FPN': smp.FPN,
    'Linknet': smp.Linknet,
    'Unet': smp.Unet
}


model_classes = conf.nclasses
if conf.mode == "ordinal":
    model_classes -= 1
model = segmentation_types[conf.segmentation_arch](conf.encoder,
                                                   classes=model_classes,
                                                   activation='sigmoid')
preprocess_fn = get_preprocessing_fn(conf.encoder)

model = model.cuda()

In [ ]:
import random
from xv import dataset

random.seed(hash("😂"))


all_files = glob(f'{train_dir}/labels/*{conf.data_prefix}_disaster.json')
random.shuffle(all_files)

dev_ix = int(len(all_files)*.20)
dev_files = all_files[:dev_ix]
train_files = all_files[dev_ix:]

train_instances = dataset.get_instances(train_files, filter_none=conf.filter_none)

dev_instances = dataset.get_instances(dev_files, filter_none=conf.filter_none)

len(train_instances), len(dev_instances)

In [ ]:
train_dataset = dataset.BuildingSegmentationDataset(
    instances=train_instances,
    nclasses=conf.nclasses,
    resolution=conf.training_resolution,
    augment=augment,
    preprocess_fn=preprocess_fn,
    mode=conf.mode,
)

dev_dataset = dataset.BuildingSegmentationDataset(
    instances=dev_instances,
    nclasses=conf.nclasses,
    resolution=conf.training_resolution,
    augment=None,
    preprocess_fn=preprocess_fn,
    mode=conf.mode,
)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=conf.batch_size,
    shuffle=True,
    num_workers=10,
)

dev_loader = torch.utils.data.DataLoader(
    dev_dataset,
    batch_size=conf.batch_size,
    shuffle=False,
    num_workers=10,
)

In [ ]:
from xv.nn.losses import loss_dict, WeightedLoss
from torch.nn.modules.loss import CrossEntropyLoss


loss = WeightedLoss({loss_dict[l]():w for l, w in conf.loss_weights.items()})

if 'class_weight' in dict(conf):
    weights = torch.Tensor(conf.class_weight).float().cuda()
    loss = CrossEntropyLoss(weights)

In [ ]:
import apex

optims = {
    'adam': torch.optim.Adam
}

optim = optims[conf.optim](model.parameters(), lr=conf.lr)

In [ ]:
from apex import amp
model, optim = amp.initialize(model, optim, opt_level=conf.amp_opt_level);

In [ ]:
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optim, factor=conf.scheduler_factor, patience=conf.scheduler_patience
)

In [ ]:
#stats.hmean([.9471, .4547, .7014, .8176]) 0.6775160778620278

In [ ]:
best_score = 0
epoch = 0

In [ ]:
from xv import run

train_fn = run.train_segment if conf.nclasses == 1 else run.train_damage
eval_fn = run.evaluate_segment if conf.nclasses == 1 else run.evaluate_damage

for epoch in range(epoch, conf.epochs):
    metrics = {'epoch': epoch}
    train_metrics = train_fn(model, optim, train_loader, loss, mode=conf.mode)
    metrics.update(train_metrics)
    
    dev_metrics = eval_fn(model, dev_loader, loss, mode=conf.mode)
    metrics.update(dev_metrics)
    
    examples = run.sample_masks(model, dev_instances, preprocess_fn, n=3)
    metrics['examples'] = [wandb.Image(im, caption=f'mask:{ix}') for e in examples for ix, im in enumerate(e)]
    
    wandb.log(metrics)
    scheduler.step(metrics['loss'])
    score = metrics[conf.metric]

    if score > best_score:
        torch.save(model.state_dict(), os.path.join(wandb.run.dir, "state_dict.pth"))
        best_score = score

In [ ]:
ix = 1000
i = train_dataset[ix]
images, masks = i['images'], i['masks']
image = images['post']
image = np.array(train_dataset.inverse_transform_image(image))
util.vis_im_mask(image, masks['damage'], size=(512*2, 512*2), opacity=.3);

In [ ]:
from collections import Counter
counts = Counter(len(i['pre']['features']) for i in train_dataset.instances)

In [ ]:
for i in run.sample_masks(model, dev_instances, preprocess_fn, sz=1024):
    display(i)

In [ ]:
from PIL import Image
for i in random.sample(dev_instances, 1):
    with torch.no_grad():
        img = np.array(Image.open(i['file_name']))
        model_in = preprocess_fn(img).transpose(2,0,1)
        model_in = torch.tensor(model_in)
        model_in = model_in.reshape(1, *model_in.shape)
        mask = model(model_in.cuda())
        mask = np.array((mask > 0).cpu())

In [ ]:
from xv import run
tps, fps, fns = [], [], []
model = model.eval()
threshold=0.5
with torch.no_grad():
    for image, mask in tqdm(iter(dev_loader)):
        out = model(image.to('cuda'))
        for o, m in zip(out, mask):
            tp, fp, fn = run.get_tp_fp_fn(o, m, threshold)
            tps.append(np.array(torch.tensor(tp).cpu()))
            fps.append(np.array(torch.tensor(fp).cpu()))
            fns.append(np.array(torch.tensor(fn).cpu()))
            
import pandas as pd
df = pd.DataFrame({'tp': tps, 'fp': fps, 'fn':fns})

In [ ]:
idx = (df.fp + df.fn).sort_values(ascending=False).index
df.iloc[idx].head()


from PIL import Image
from xv import util
import cv2


i = dev_instances[ix]
sz = conf.training_resolution

#def analyse_instance(i):
with torch.no_grad():
    img = np.array(Image.open(i['file_name']))
    img = cv2.resize(img, (sz,sz))
    model_in = preprocess_fn(img).transpose(2,0,1)
    model_in = torch.tensor(model_in).float()
    model_in = model_in.reshape(1, *model_in.shape)
    mask = model(model_in.cuda())
    mask = np.array((mask > 0).cpu())
    im = util.vis_im_mask(img, mask[0], opacity=.3, size=(sz,sz))
    
_, target = dev_dataset[ix]
mask = mask.astype(bool).reshape(sz,sz)
target = target.astype(bool).reshape(sz,sz)

tp = target & mask
fp = mask & ~target
fn = target & ~mask

util.vis_im_mask(img, target, opacity=.2, colours=("blue", "yellow", "red"))
util.vis_im_mask(img, np.stack((tp, fn, fp)), opacity=.2, colours=("blue", "yellow", "red"))


In [ ]:
tta_model = tta.SegmentationTTAWrapper(model, transforms, merge_mode='mean')